<a href="https://colab.research.google.com/github/pedroafleite/mlops_exercise/blob/main/pyspark/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

from google.colab import drive
drive.mount('/content/drive')

     |████████████████████████████████| 281.3 MB 46 kB/s 
     |████████████████████████████████| 198 kB 63.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=32cbf2e1dc4308cde064a14b6146f4f96cf2f6355eae95ba5d57ac174f0bc020
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
Mounted at /content/drive


In [ ]:
import pyspark

sc = pyspark.SparkContext()
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()
spark

In [ ]:
import json

with open('/content/drive/MyDrive/Colab Notebooks/pyspark/samples_output-20_06-20_08-1') as f:
    lines = f.readlines()
lines = lines[:-1]
lines.append("}")

with open('samples_output-20_06-20_08-1.json', 'w') as f:
    for item in lines:
        f.write("%s\n" % item)

with open('samples_output-20_06-20_08-1.json', 'r') as f2:
    data = json.loads(f2.read())
    print(data)

{'eventTimestamp': '2019-04-02T21:55:57Z', 'eventId': '3dc6eb9af4154635b688e7cfeb8db131bfe84ccd', 'eventSource': 'doc.contract', 'eventVersion': '1.0', 'eventType': 'DocSchedule', 'doc': {'schemaVersion': '1.0', 'docId': 'CF2909E5-12C8-4D42-A996-12672B727B3B', 'metadata': {'accountId': 'EA2425CF-EF72-40C6-8DD3-C7D5310FA07D', 'superTag01': 'A', 'superTag02': 'C', 'otherTags': [{'tagName': 'otherTag01', 'tagValue': 'A'}, {'tagName': 'otherTag02', 'tagValue': 'X'}]}, 'payload': 'texto teste'}}


In [ ]:
data['eventTimestamp']

'2019-04-02T21:55:57Z'

In [ ]:
import pandas as pd

pandas_df = pd.json_normalize(data)
pandas_df

,eventTimestamp,eventId,eventSource,eventVersion,eventType,doc.schemaVersion,doc.docId,doc.metadata.accountId,doc.metadata.superTag01,doc.metadata.superTag02,doc.metadata.otherTags,doc.payload
0,2019-04-02T21:55:57Z,3dc6eb9af4154635b688e7cfeb8db131bfe84ccd,doc.contract,1.0,DocSchedule,1.0,CF2909E5-12C8-4D42-A996-12672B727B3B,EA2425CF-EF72-40C6-8DD3-C7D5310FA07D,A,C,"[{'tagName': 'otherTag01', 'tagValue': 'A'}, {...",texto teste


In [ ]:
print(
    data['eventTimestamp'],
    data['eventSource'],
    data['eventType'],
    data['eventVersion'],
    data['doc']['metadata']['accountId'],
    data['doc']['metadata']['superTag01'],
    data['doc']['metadata']['superTag02'],
    data['doc']['metadata']['otherTags'],
    other_tags,
    data['doc']['payload']
)

2019-04-02T21:55:57Z doc.contract DocSchedule 1.0 EA2425CF-EF72-40C6-8DD3-C7D5310FA07D A C [{'tagName': 'otherTag01', 'tagValue': 'A'}, {'tagName': 'otherTag02', 'tagValue': 'X'}] texto teste


In [ ]:
list_tags = data['doc']['metadata']['otherTags']

d = []

for i in range(len(list_tags)):
  a = data['doc']['metadata']['otherTags'][i]['tagName']
  b = data['doc']['metadata']['otherTags'][i]['tagValue']
  c = a + ':' + b
  d.append(c)

other_tags = '|'.join(d)
print(other_tags)

otherTag01:A|otherTag02:X


In [ ]:
import pandas as pd

pandas_df = pd.DataFrame({
    'eventTimestamp': data['eventTimestamp'],
    'eventSource': data['eventSource'],
    'eventType': data['eventType'],
    'eventVersion': data['eventVersion'],
    'accountId': data['doc']['metadata']['accountId'],
    'superTag01': data['doc']['metadata']['superTag01'],
    'superTag02': data['doc']['metadata']['superTag02'],
    'concatOtherTags': other_tags,
    'payload': data['doc']['payload']
})
pandas_df

ValueError: ignored

In [ ]:
import pandas as pd

pandas_df = pd.DataFrame({
    'eventTimestamp': [data['eventTimestamp']],
    'eventSource': [data['eventSource']],
    'eventType': [data['eventType']],
    'eventVersion': [data['eventVersion']],
    'accountId': [data['doc']['metadata']['accountId']],
    'superTag01': [data['doc']['metadata']['superTag01']],
    'superTag02': [data['doc']['metadata']['superTag02']],
    'concatOtherTags': [other_tags],
    'payload': [data['doc']['payload']]
})
df = spark.createDataFrame(pandas_df)
df.show()

+--------------------+------------+-----------+------------+--------------------+----------+----------+--------------------+-----------+
|      eventTimestamp| eventSource|  eventType|eventVersion|           accountId|superTag01|superTag02|     concatOtherTags|    payload|
+--------------------+------------+-----------+------------+--------------------+----------+----------+--------------------+-----------+
|2019-04-02T21:55:57Z|doc.contract|DocSchedule|         1.0|EA2425CF-EF72-40C...|         A|         C|otherTag01:A|othe...|texto teste|
+--------------------+------------+-----------+------------+--------------------+----------+----------+--------------------+-----------+



In [ ]:
df.write.parquet("/content/drive/MyDrive/Colab Notebooks/pyspark/output/proto.parquet")